<DIV ALIGN=CENTER>

# Introduction to NLP: Semantic Analysis
## Professor Robert J. Brunner
  
</DIV>  
-----
-----


## Introduction

Semantic Analysis
Derive meaning from words, chunks, tokens.

word2vec, doc2vec

gensim

-----



-----

## Student Activity

In the preceding cells, we . Now that you have run the Notebook,
go back and make the following changes to see how the results change.

1. Change 
2. Try 
3. Can 

-----